In [1]:
# https://docs.bittensor.com/subnets/register-validate-mine#check-the-permit-status
"""
Except in Subnet 1, all subnets have 256 UID slots per subnet. Of these 256 UID slots,
a subnet can have a maximum of 64 subnet validator UIDs and 192 (i.e., 256-64) UIDs for subnet miners.
"""

import bittensor as bt
import asyncio

NET_UID = 61
NETWORK = "test"
WALLET = "validator"

In [2]:
# Initialize your wallet
wallet = bt.wallet(name=WALLET)

# Initialize the subtensor
subtensor = bt.subtensor(network=NETWORK)

# Fetch the metagraph
subnet = subtensor.metagraph(netuid=NET_UID)

# Replace below with your SS58 hotkey 
hotkey = wallet.hotkey.ss58_address
my_uid = subnet.hotkeys.index(wallet.hotkey.ss58_address)
sub = bt.subtensor(NETWORK)
mg = sub.metagraph(NET_UID)
if hotkey not in mg.hotkeys:
    print(f"Hotkey {hotkey} deregistered")
else:
    print(f"Hotkey {hotkey} is registered. UID: {my_uid}")

Hotkey 5Fk35HgrTqqUffK7WN8FG4euZ8MpKx35mUYz9kgwj3UDnNHr is registered. UID: 166


In [3]:
len(subnet.validator_permit)

204

In [4]:
# Extract active UIDs
active_uids = subnet.uids.tolist()

# Print the active UIDs
print(f"Active UIDs (total: {len(active_uids)}): {active_uids}")
print(f"My hotkey is active: {my_uid in active_uids}")

Active UIDs (total: 204): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203]
My hotkey is active: True


In [5]:
subnet.validator_permit[my_uid]

False

In [6]:
subnet.validator_permit.shape # 1000+ tao

(204,)

In [7]:
# Sample validators.
# Define your request payload (e.g., a text prompting request)
# import time
from prompting.protocol import StreamPromptingSynapse
# request_payload = bt.synapse.TextPrompting(
#     text="Hello, can you process this request?"
# )
# request_payload

# Get the list of uids to query for this step.
# uids = get_random_uids(self, k=k, exclude=exclude or []).to(self.device)
# uids = get_random_uids(self, k=k, exclude=exclude or []).to(self.device)
# uids_cpu = uids.cpu().tolist()

uids = [my_uid]
axons = [subnet.axons[uid] for uid in uids]

# # Directly call dendrite and process responses in parallel
dendrite = bt.dendrite(wallet=wallet)
roles = ["user"]
messages = ["Capital of Zimbabwe?"]
timeout = 17
streams_responses = await dendrite(
    axons=axons,
    synapse=StreamPromptingSynapse(roles=roles, messages=messages),
    timeout=timeout,
    deserialize=False,
    streaming=True,
)
# Prepare the task for handling stream responses
from prompting.dendrite import DendriteResponseEvent
from prompting.forward import handle_response
from transformers import AutoTokenizer

stream_results_dict = dict(zip(uids, streams_responses))
model_name = "casperhansen/llama-3-8b-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)
handle_stream_responses_task = asyncio.create_task(handle_response(stream_results_dict, tokenizer))
stream_results = await handle_stream_responses_task
# if not agent.task.static_reference:
#     reference_generation_task = generate_reference(agent)
#     _, stream_results = await asyncio.gather(
#         reference_generation_task, handle_stream_responses_task
#     )
# else:
#     stream_results = await handle_stream_responses_task

# log_stream_results(stream_results)

# Encapsulate the responses in a response event (dataclass)
response_event = DendriteResponseEvent(
    stream_results=stream_results, uids=uids, timeout=timeout
)

bt.logging.info(f"Created DendriteResponseEvent:\n {response_event}")

/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-18 13:30:49,105	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:bittensor: - Created DendriteResponseEvent:
 DendriteResponseEvent(uids=[166], completions=[''], timings=[17], status_messages=['Timedout after 17.0 seconds.'], status_codes=[408]) - 


2024-06-18 13:31:25.474 |       INFO       |  - Created DendriteResponseEvent:
 DendriteResponseEvent(uids=[166], completions=[''], timings=[17], status_messages=['Timedout after 17.0 seconds.'], status_codes=[408]) - 


In [8]:

# Reward the responses and get the reward result (dataclass)
# This contains a list of RewardEvents but can be exported as a dict (column-wise) for logging etc
# reward_result = RewardResult(
#     self.reward_pipeline,
#     agent=agent,
#     response_event=response_event,
#     device=self.device,
# )
# bt.logging.info(f"Created RewardResult:\n {reward_result}")

# best_response = response_event.completions[reward_result.rewards.argmax()]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:bittensor: - Created DendriteResponseEvent:
 DendriteResponseEvent(uids=[166], completions=[''], timings=[0], status_messages=['Service at 194.68.245.74:8091/StreamPromptingSynapse unavailable.'], status_codes=[503]) - 


2024-06-17 11:39:42.019 |       INFO       |  - Created DendriteResponseEvent:
 DendriteResponseEvent(uids=[166], completions=[''], timings=[0], status_messages=['Service at 194.68.245.74:8091/StreamPromptingSynapse unavailable.'], status_codes=[503]) - 


In [53]:
stream_results

[SynapseStreamResult(exception=UnboundLocalError("local variable 'chunk' referenced before assignment"), uid=166, accumulated_chunks=[], accumulated_chunks_timings=[], tokens_per_chunk=[], synapse=StreamPromptingSynapse(required_hash_fields=['messages'], roles=['user'], messages=['failure'], completion=''))]

In [44]:
response_event

DendriteResponseEvent(uids=[166], completions=[''], timings=[0], status_messages=['Service at 194.68.245.74:8091/StreamPromptingSynapse unavailable.'], status_codes=[503])

In [35]:
streams_responses

[<async_generator object DendriteMixin.call_stream at 0x7f2dc3d30fc0>]

In [ ]:
# Function to send a request to a single validator
async def send_request_to_validator(dendrite, uid):
    axon_info = subnet.axons[uid.item()]
    response = await dendrite.query(
        axons=[axon_info],
        synapse=request_payload,
        timeout=17  # Adjust timeout as needed
    )
    return response

# Function to send requests to all validators
async def send_requests_to_all_validators(dendrite, active_uids):
    tasks = [send_request_to_validator(dendrite, uid) for uid in active_uids]
    responses = await asyncio.gather(*tasks, return_exceptions=True)
    return responses

# Initialize Dendrite
dendrite = bt.dendrite(wallet=wallet)

# Run the function to send requests
responses = asyncio.run(send_requests_to_all_validators(dendrite, active_uids))

# Process responses
for response in responses:
    if isinstance(response, Exception):
        print(f"Request failed with exception: {response}")
    else:
        print(f"Received response: {response}")
